<a href="https://colab.research.google.com/github/fboldt/aulasann/blob/main/aula05e_real_world_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [171]:
import numpy as np

def include_bias(X):
  bias = np.ones((X.shape[0],1))
  Xb = np.concatenate((bias, X), axis=1)
  return Xb

class Backpropagation():
  def __init__(self, n_hidden=[2], max_iter=1000, learning_rate=0.01):
    self.n_hidden = n_hidden
    self.max_iter = max_iter
    self.learning_rate = learning_rate

  def foward(self, X):
    self.A = []
    self.Z = []
    AUX = X.copy()
    for W in self.Ws:
      self.A.append(include_bias(AUX))
      self.Z.append(self.A[-1] @ W)
      AUX = np.tanh(self.Z[-1])
    return AUX

  def backward(self, y, ypred):
    grads = []
    output_delta = y - ypred
    grads.insert(0, self.A[-1].T @ output_delta)
    for i in range(len(self.Ws)-1, 0, -1):
      tanh_grad = (1 - np.tanh(self.Z[i-1])**2)
      input_delta = (output_delta @ self.Ws[i][1:, :].T) * tanh_grad
      grads.insert(0, self.A[i-1].T @ input_delta)
      output_delta = input_delta.copy()
    for i in range(len(self.Ws)-1, -1, -1):
      self.Ws[i] += grads[i] * self.learning_rate

  def get_ws(self, X, y):
    self.Ws = []
    previous_output = X.shape[1]
    for n in self.n_hidden:
      self.Ws.append(np.random.uniform(-1, 1, size=(previous_output+1, n)))
      previous_output = n
    self.Ws.append(np.random.uniform(-1, 1, size=(previous_output+1, y.shape[1])))
    for _ in range(self.max_iter):
      ypred = self.foward(X)
      self.backward(y, ypred)
    return self.Ws

In [179]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelBinarizer

def sign(x):
  return np.sign(x)

class MLP(BaseEstimator, ClassifierMixin):
  def __init__(self, training_algorithm=Backpropagation()):
    self.training_algorithm = training_algorithm

  def fit(self, X, y):
    self.lb = LabelBinarizer()
    y = self.lb.fit_transform(y)
    y = y * 2 - 1
    self.w = self.training_algorithm.get_ws(X, y)
    return self

  def predict(self, X):
    ypred = self.training_algorithm.foward(X)
    return self.lb.inverse_transform(ypred)

model = MLP(Backpropagation([], max_iter=5000, learning_rate=0.0001))
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(accuracy_score(y_test, ypred))

0.9444444444444444


In [180]:
from sklearn.model_selection import cross_validate

scores = cross_validate(model, X, y, cv=5, scoring='accuracy')
print(scores['test_score'])
print(np.mean(scores['test_score']))

[0.90833333 0.86388889 0.90807799 0.93314763 0.86629526]
0.8959486227174249
